In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,LabelEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [63]:
df=pd.read_csv('Datasets/titanic.csv')
df.head(2)

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [64]:
df=df.drop(columns=['Cabin','PassengerId','Ticket','Name'])
df=df.iloc[:,1:]

In [65]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [60]:
le=LabelEncoder()
temp=le.fit_transform(df['Embarked'])

In [61]:
temp[0:5]

array([2, 0, 2, 2, 2])

In [27]:
from sklearn.preprocessing import OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first'),['Sex']),
        ('onehot_embark',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),['Embarked'])
        
    ],
    remainder='passthrough'  # Include the remaining columns as is
)

In [28]:
temp=preprocessor.fit_transform(df)

In [33]:
temp[0:5]

array([[ 1.    ,  1.    ,  0.    ,  0.    ,  3.    , 22.    ,  1.    ,
         0.    ,  7.25  ],
       [ 0.    ,  0.    ,  0.    ,  1.    ,  1.    , 38.    ,  1.    ,
         0.    , 71.2833],
       [ 0.    ,  1.    ,  0.    ,  1.    ,  3.    , 26.    ,  0.    ,
         0.    ,  7.925 ],
       [ 0.    ,  1.    ,  0.    ,  1.    ,  1.    , 35.    ,  1.    ,
         0.    , 53.1   ],
       [ 1.    ,  1.    ,  0.    ,  0.    ,  3.    , 35.    ,  0.    ,
         0.    ,  8.05  ]])

In [21]:
preprocessor.named_transformers_ .onehot.categories_

[array(['female', 'male'], dtype=object),
 array(['C', 'Q', 'S', nan], dtype=object)]

In [32]:
temp=np.delete(temp,[1],axis=1)

In [66]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=.2,random_state=42)

In [67]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 331 to 102
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    object 
 2   Age       572 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


In [68]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [69]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [25]:
onehotencoder = OneHotEncoder(categories='auto',drop='first',sparse_output =False)
feature = onehotencoder.fit(df.iloc[:, 2].values.reshape(-1,1),)

# Making each step transformations

## missing_values -> categorical encoding -> Scaling -> k best -> train

In [70]:
# missing values
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_class',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [71]:
# encoding
trf2=ColumnTransformer([
    ('le_sex_embarked', LabelEncoder (),[1,6])    
],remainder='passthrough')

In [78]:
trf3=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),[1]),
    ('ohe_embarked',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),[6])
],remainder='passthrough')

In [79]:
# scaling
trf4=ColumnTransformer([
    ('scaling_all',MinMaxScaler(),slice(0,8))
],remainder='passthrough')


In [80]:
# feature selection
trf4=SelectKBest(score_func=chi2,k=5)

In [81]:
# ML algo
trf6=DecisionTreeClassifier()

# Create Pipeline

In [82]:
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

In [84]:
pipe.fit(X_train,y_train)

TypeError: LabelEncoder.fit_transform() takes 2 positional arguments but 3 were given

In [1]:
# display pipeline
from sklearn import set_config
set_config(display='diagram')